In [1]:
!pip --version

pip 24.2 from /home/antonio/.local/lib/python3.10/site-packages/pip (python 3.10)


In [2]:
!pip install pyspark py4j

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 7.9 MB/s eta 0:00:00m eta 0:00:010:00:02
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812371 sha256=38f6c93c4fbd5b987da6abb423f9abe7704fa41be3a3ebef16ce36e2e8a2b585
  Stored in directory: /home/antonio/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [30]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import col, round

conf = (
     SparkConf()
    .setAppName('Simple_Spark')
    .setMaster('local[*]')
)


# Spark сессия
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    # .enableHiveSupport()
    .getOrCreate()
)

In [15]:
# создать DF
data = [(1, "Moscow", 15), (2, "Rio", 5), (3, "GonKong", 10)]
columns = ["id", "city", "population"]

df =  spark.createDataFrame(data).toDF(*columns)
df.show()

+---+-------+----------+
| id|   city|population|
+---+-------+----------+
|  1| Moscow|        15|
|  2|    Rio|         5|
|  3|GonKong|        10|
+---+-------+----------+



In [22]:
# создать DF c RDD

columns2 = ["language","users_count"]
data2 = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

rdd = spark.sparkContext.parallelize(data2)

df2 = rdd.toDF(columns2)

df2.show()

df2.printSchema()

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

root
 |-- language: string (nullable = true)
 |-- users_count: string (nullable = true)



In [26]:
# создать RDD с помощью схемы

from pyspark.sql.types import StructField, StructType, StringType, IntegerType

columns3 = ["language","users_count"]
data3 = [("Java", 20000, 3000), ("Python", 100000, 2500), ("Scala", 3000, 3500), ("JavaScript", 5000, 2000), ("Kotlin", 4000, 2500)]

schema = StructType([
    StructField("language", StringType(), True),
    StructField("users_count", IntegerType(), True),
    StructField("avg_sal", IntegerType(), True)
])

df3 = spark.createDataFrame(data3, schema)

df3.show()

df3.printSchema()


+----------+-----------+-------+
|  language|users_count|avg_sal|
+----------+-----------+-------+
|      Java|      20000|   3000|
|    Python|     100000|   2500|
|     Scala|       3000|   3500|
|JavaScript|       5000|   2000|
|    Kotlin|       4000|   2500|
+----------+-----------+-------+

root
 |-- language: string (nullable = true)
 |-- users_count: integer (nullable = true)
 |-- avg_sal: integer (nullable = true)



In [37]:
new_df3 = (
            df3
            .withColumn("avg_sal", round((col("avg_sal") * 0.2) + col("avg_sal")).cast("Int"))
            .select(col("language"),
                       (col("users_count") + 500).alias("new_user_count"),
                    col("avg_sal")
                
            )
        )
new_df3.printSchema()
new_df3.show()

root
 |-- language: string (nullable = true)
 |-- new_user_count: integer (nullable = true)
 |-- avg_sal: integer (nullable = true)

+----------+--------------+-------+
|  language|new_user_count|avg_sal|
+----------+--------------+-------+
|      Java|         20500|   3600|
|    Python|        100500|   3000|
|     Scala|          3500|   4200|
|JavaScript|          5500|   2400|
|    Kotlin|          4500|   3000|
+----------+--------------+-------+



In [42]:
spark.stop()

''